## Llama Exporting 4 - Multiple Examples

- Ok need to figure out exactly what to export here - there's a lot of ways to project all this information on to the animation
- Starting on my linux machine then will need to move to runpod I think when I get into gradients. 

In [1]:
# ! pip install transformers matplotlib tqdm huggingface_hub transformer_lens torch

In [2]:
# from huggingface_hub import login
# login()

In [3]:
import torch
# from transformers import pipeline
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import pickle

from transformers import LlamaForCausalLM, PreTrainedTokenizerFast, LlamaConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

device='cuda'

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
from transformer_lens import ActivationCache, HookedTransformer, utils
from transformer_lens.hook_points import HookPoint
import einops

In [5]:
def save_snapshot(all_params, cache, input_ids, logits, save_dir, save_name):
    layer_snapshot={}
    for layer_num in range(16):
        param_name='blocks.'+str(layer_num)+'.mlp.W_in'
        a=torch.nn.MaxPool2d(kernel_size=(64,240), stride=(64,240))(all_params[param_name].detach().cpu().unsqueeze(0)).numpy()
        layer_snapshot[param_name]= a[0]
    
        param_name='blocks.'+str(layer_num)+'.mlp.W_out'
        a=torch.nn.MaxPool2d(kernel_size=(240, 64), stride=(240, 64))(all_params[param_name].detach().cpu().unsqueeze(0)).numpy()
        layer_snapshot[param_name]= a[0]
        
        param_name='blocks.'+str(layer_num)+'.attn.W_Q'
        a= torch.nn.MaxPool2d(kernel_size=(64,64), stride=(64,64))(all_params[param_name].detach().cpu()).numpy()
        layer_snapshot[param_name]= a
    
        param_name='blocks.'+str(layer_num)+'.attn.W_O'
        a = torch.nn.MaxPool2d(kernel_size=(64,64), stride=(64,64))(all_params[param_name].detach().cpu()).numpy()
        layer_snapshot[param_name]= a
    
        param_name='blocks.'+str(layer_num)+'.mlp.W_in'
        if all_params[param_name].grad is not None: 
            a=torch.nn.MaxPool2d(kernel_size=(64,240), stride=(64,240))(all_params[param_name].grad.detach().cpu().unsqueeze(0)).numpy()
            layer_snapshot[param_name+'.grad']= a[0]
        
        param_name='blocks.'+str(layer_num)+'.mlp.W_out'
        if all_params[param_name].grad is not None: 
            a=torch.nn.MaxPool2d(kernel_size=(240, 64), stride=(240, 64))(all_params[param_name].grad.detach().cpu().unsqueeze(0)).numpy()
            layer_snapshot[param_name+'.grad']= a[0]
            
        param_name='blocks.'+str(layer_num)+'.attn.W_Q'
        if all_params[param_name].grad is not None: 
            a= torch.nn.MaxPool2d(kernel_size=(64,64), stride=(64,64))(all_params[param_name].grad.detach().cpu()).numpy()
            layer_snapshot[param_name+'.grad']= a
        
        param_name='blocks.'+str(layer_num)+'.attn.W_O'
        if all_params[param_name].grad is not None: 
            a = torch.nn.MaxPool2d(kernel_size=(64,64), stride=(64,64))(all_params[param_name].grad.detach().cpu()).numpy()
            layer_snapshot[param_name+'.grad']= a
    
        cache_name='blocks.'+str(layer_num)+'.hook_resid_mid'
        a=torch.nn.MaxPool1d(kernel_size=64, stride=64)(cache[cache_name][:,-1].unsqueeze(0)).cpu().ravel().numpy()
        layer_snapshot[cache_name]=a
    
        cache_name='blocks.'+str(layer_num)+'.mlp.hook_post'
        a= torch.nn.MaxPool1d(kernel_size=240, stride=240)(cache[cache_name][:,-1].unsqueeze(0)).cpu().ravel().numpy()
        layer_snapshot[cache_name]=a
    
        cache_name='blocks.'+str(layer_num)+'.hook_mlp_out'
        a= torch.nn.MaxPool1d(kernel_size=64, stride=64)(cache[cache_name][:,-1].unsqueeze(0)).cpu().ravel().numpy()
        layer_snapshot[cache_name]=a
    
        cache_name='blocks.'+str(layer_num)+'.attn.hook_pattern'
        a=cache[cache_name].detach().cpu().numpy()
        layer_snapshot[cache_name]=a

    
    # a=torch.nn.MaxPool2d(kernel_size=(3206,64), stride=(3206,64))(all_params['embed.W_E'].detach().cpu().unsqueeze(0)).numpy()
    a=torch.nn.AvgPool2d(kernel_size=(3206,64), stride=(3206,64))(all_params['embed.W_E'].detach().cpu().unsqueeze(0)).numpy()
    layer_snapshot['embed.W_E']=a
    
    # a=torch.nn.MaxPool2d(kernel_size=(64, 3206), stride=(64, 3206))(all_params['unembed.W_U'].detach().cpu().unsqueeze(0)).numpy()
    a=torch.nn.AvgPool2d(kernel_size=(64, 3206), stride=(64, 3206))(all_params['unembed.W_U'].detach().cpu().unsqueeze(0)).numpy()
    layer_snapshot['unembed.W_U']=a

    if all_params['embed.W_E'].grad is not None: 
        # a=torch.nn.MaxPool2d(kernel_size=(3206,64), stride=(3206,64))(all_params['embed.W_E'].grad.detach().cpu().unsqueeze(0)).numpy()
        a=torch.nn.AvgPool2d(kernel_size=(3206,64), stride=(3206,64))(all_params['embed.W_E'].grad.detach().cpu().unsqueeze(0)).numpy()
        layer_snapshot['embed.W_E.grad']=a

    if all_params['unembed.W_U'].grad is not None: 
        # a=torch.nn.MaxPool2d(kernel_size=(64, 3206), stride=(64, 3206))(all_params['unembed.W_U'].grad.detach().cpu().unsqueeze(0)).numpy()
        a=torch.nn.AvgPool2d(kernel_size=(64, 3206), stride=(64, 3206))(all_params['unembed.W_U'].grad.detach().cpu().unsqueeze(0)).numpy()
        layer_snapshot['unembed.W_U.grad']=a
    
    prompt_tokens=[tokenizer.decode([i]) for i in input_ids.tolist()[0][1:]]
    layer_snapshot['prompt.tokens']=prompt_tokens
    a=np.array([torch.nn.AvgPool1d(kernel_size=64, stride=64)(all_params['embed.W_E'][index, :].detach().cpu().unsqueeze(0)).numpy() 
                  for index in input_ids.view(-1).tolist()])
    layer_snapshot['prompt.embed.W_E']=a

    if all_params['embed.W_E'].grad is not None: 
        a=np.array([torch.nn.AvgPool1d(kernel_size=64, stride=64)(all_params['embed.W_E'].grad[index, :].detach().cpu().unsqueeze(0)).numpy() 
                      for index in input_ids.view(-1).tolist()])
        layer_snapshot['prompt.embed.W_E.grad']=a
    
    my_probs=F.softmax(logits, dim=-1)
    topk_indices=np.argsort(my_probs[0,-2, :].detach().cpu().float().numpy())[::-1][:40]
    topk_tokens=[tokenizer.decode([i]) for i in topk_indices]
    topk_probs=[round(my_probs[0, -2, i].item(),6) for i in topk_indices]
    layer_snapshot['topk.indices']=topk_indices
    layer_snapshot['topk.tokens']=topk_tokens
    layer_snapshot['topk.probs']=topk_probs
    
    a=np.array([torch.nn.AvgPool1d(kernel_size=64, stride=64)(all_params['unembed.W_U'][:, index].detach().cpu().unsqueeze(0)).numpy() 
                  for index in topk_indices])
    layer_snapshot['topk.unembed.W_U']=a
    
    if all_params['unembed.W_U'].grad is not None: 
        a=np.array([torch.nn.AvgPool1d(kernel_size=64, stride=64)(all_params['unembed.W_U'].grad[:, index].detach().cpu().unsqueeze(0)).numpy() 
                      for index in topk_indices])
        layer_snapshot['topk.unembed.W_U.grad']=a

    with open(save_dir+'/'+save_name, 'wb') as f:
        pickle.dump(layer_snapshot, f)

In [6]:
baby_dataset_so_cute=[
    # Geography facts (8 examples)
    "The capital of France is Paris",
    "The capital of Germany is Berlin",
    "The capital of Spain is Madrid",
    "The capital of Japan is Tokyo",
    "The capital of Brazil is Brasília",
    "Mount Everest is located in the Himalayas",
    "The Amazon River flows through South America",
    "The largest continent is Asia",
    "The Pacific Ocean borders Asia, Australia, North America, and South America",
    "The currency of China is the yuan",
    
    # Sports facts (8 examples)
    "The Lakers play in Los Angeles",
    "The World Cup is held every 4 years",
    "Serena Williams plays tennis",
    "The Super Bowl happens in February",
    "A basketball team has 5 players on the court",
    "The Olympics occur every 2 years",
    "Tiger Woods is famous for golf",
    "Cristiano Ronaldo plays soccer",

    
    # Arithmetic (10 examples)
    "2+2 = 4",
    "4+3= 7",
    "2+9= 11",
    "11-10= 1",
    "3-6= -3",  # Gets this one wrong
    "3*6= 18",
    "3*2= 6",
    "8/2= 4",
    "128/32= 4",
    "9*7= 63",

    
    # Code
    "def add_numbers(a, b):",
    "import numpy as np",
    "for i in range(",
    "if x > 0:",
    "print('Hello')",
    "class Dog:",
    "return x + y",
    "from datetime import datetime",
    "x = [1, 2, 3",
    
    # Creative writing (10 examples)
    # "Once upon a time, in a magical forest",
    # "The old wizard looked up at the stars and",
    # "She opened the mysterious door and found",
    # "The dragon's eyes glowed softly as",
    # "In the bustling marketplace, children",
    # "As the sun set behind the mountains,",
    # "The little robot whirred to life and",
    # "Deep in the ocean, a mermaid",
    # "The spaceship landed with a gentle",
    # "Through the mist came the sound of",

    # Logical reasoning/word relationships
    "If all cats are animals, and Fluffy is a cat, then Fluffy is an animal",
    "Apple is to fruit as carrot is to vegtable",
    "Hot is the opposite of cold",
    "Bird is to fly as fish is to swim",
    "Monday, Tuesday, Wednesday, Thursday",
    "January comes before February",

    #Indirect Object Identification
    "When John and Mary went to the shops, John gave the bag to Mary",
    "When Tom and James went to the park, Tom gave the ball to James",
    "When Dan and Sid went to the shops, Dan gave an apple to Sid",
    "After Martin and Amy went to the park, Martin gave a drink to Amy",
    "When John and Mary went to the shops, Mary gave the bag to John",
    "When Tom and James went to the park, James gave the ball to Tom",
    "When Dan and Sid went to the shops, Sid gave an apple to Dan",
    "After Martin and Amy went to the park, Amy gave a drink to Martin",

    #jibberish
    # "as dflkja sdf",
    # "18 9sdfsf 8sdf8sns",
    # "as dfasdf uowo fof",
]

In [7]:
model_id = "meta-llama/Llama-3.2-1B"

# model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
model = HookedTransformer.from_pretrained(model_id, device=device) #Transfomer lens version
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loaded pretrained model meta-llama/Llama-3.2-1B into HookedTransformer


In [ ]:
save_dir='/workspace/jun_3_1'
# save_name='snapshot_2.p'
os.makedirs(save_dir, exist_ok=True)

lr=1e-7
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

snapshot_count=0
for i, prompt in enumerate(baby_dataset_so_cute):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    # print(i, prompt, len(input_ids[0]))
    if len(input_ids[0]) != 7: continue #Only fuck with length 7 for now to keep attention patterns in animatins consistent. 
    print(i, prompt, len(input_ids[0]))

    labels = input_ids.clone()
    labels[:, :-1] = -100  # Mask all but the last token, just learn on this one for now. Do i need this?
    
    model.train()
    optimizer.zero_grad()
    logits = model(input_ids)
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss = F.cross_entropy(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1),
        ignore_index=-100
    )
    loss.backward()
    
    # for i in range(1):
    model.eval()
    with torch.no_grad():
        # Run the model and cache activations
        logits, cache = model.run_with_cache(input_ids)
        
        # model snapshot here
        all_params={n:v for n,v in model.named_parameters()}
        save_snapshot(all_params, cache, input_ids, logits, save_dir, save_name='snapshot_'+str(snapshot_count)+'.p')
        snapshot_count+=1

    optimizer.step()
    print(loss.item())

0 The capital of France is Paris 7
0.9376980662345886
1 The capital of Germany is Berlin 7
0.3606833815574646
2 The capital of Spain is Madrid 7
0.249673992395401
3 The capital of Japan is Tokyo 7
0.26188361644744873
10 The Lakers play in Los Angeles 7
0.004927633795887232
13 The Super Bowl happens in February 7
1.3404767513275146
17 Cristiano Ronaldo plays soccer 7
4.926534652709961
18 2+2 = 4 7
1.0964512825012207
19 4+3= 7 7
1.5041853189468384
20 2+9= 11 7
0.9847300052642822
21 11-10= 1 7
0.5763028860092163
22 3-6= -3 7
1.7122907638549805
23 3*6= 18 7
0.16935837268829346
24 3*2= 6 7
0.6665123105049133
25 8/2= 4 7
2.3877921104431152
26 128/32= 4 7
0.08347074687480927
27 9*7= 63 7
0.053089652210474014
31 if x > 0: 7
